<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/123_Logging_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧱 Logging Setup Plan

1. **Define** the `setup_logging()` function at the top
2. **Call** `setup_logging()` immediately after
3. Then begin building out the rest of your agent code

---

## 🔍 Breakdown: What Each Part Does

```python
def setup_logging(log_file="agent_log.txt", level=logging.INFO):
```

* Lets you **customize** the log file and verbosity level
* `level=logging.INFO` means you'll capture INFO, WARNING, ERROR, etc.

---

```python
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
```

* This clears old logging handlers.
* It's **crucial in Jupyter or Colab** environments, because logging configs **persist across cells**. Without this, re-running cells can result in **duplicate logs or no output at all**.

---

```python
logging.basicConfig(
    filename=log_file,
    level=level,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filemode="a"  # append to file; use "w" to overwrite on each run
)
```

* ✅ Saves logs to a file (`agent_log.txt`)
* ✅ Includes timestamp, level, and message
* 🔄 `filemode="a"` makes it append rather than overwrite — great for multi-step runs

---

## 🧪 When to Use This Setup

* ✅ At the start of any notebook or script involving:

  * **Agents**
  * **Tool calls**
  * **Retries**
  * **Debugging**
* ✅ Especially useful in **iterative notebooks** where you're:

  * Debugging failures
  * Building multi-step processes
  * Logging durations, retries, or exceptions



In [3]:
import logging

def setup_logging(log_file="agent_log.txt", level=logging.INFO):
    import logging

    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=level,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()  # prints to notebook output
        ]
    )

setup_logging()

✅ — writing **intentionally buggy code**, logging the errors, and then **debugging from the logs** is one of the *most powerful ways* to build your error-handling intuition.

---

### 🧪 Step-by-Step Plan

We'll go through this loop:

1. **Write buggy code**
2. **Wrap it with error handling**
3. **Log the error**
4. **Run it, inspect `agent_log.txt`**
5. **Fix the bug**
6. Repeat (with a slightly more complex example)




### 🧩 Example 1: TypeError — Adding string to int





In [4]:
def buggy_add(x, y):
    return x + y

try:
    result = buggy_add(5, "hello")  # TypeError!
except Exception as e:
    logging.exception("Error in buggy_add")  # logs traceback

2025-09-08 18:43:23,950 [ERROR] Error in buggy_add
Traceback (most recent call last):
  File "/tmp/ipython-input-945670281.py", line 5, in <cell line: 0>
    result = buggy_add(5, "hello")  # TypeError!
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-945670281.py", line 2, in buggy_add
    return x + y
           ~~^~~
TypeError: unsupported operand type(s) for +: 'int' and 'str'


### 🛠️ Now Let’s Fix It

You realize that `x` and `y` should both be numbers. So, update the function like this:

---

### 🧠 What You Just Practiced

* Raising specific exceptions (`TypeError`)
* Catching with `except`
* Logging with `logging.exception` (includes traceback)
* Using the logs to *fix* the bug
* Writing defensive code with type checks


In [5]:
def fixed_add(x, y):
    if not all(isinstance(i, (int, float)) for i in [x, y]):
        raise TypeError("Both arguments must be int or float")
    return x + y

try:
    result = fixed_add(5, "hello")
except Exception as e:
    logging.exception("Error in fixed_add")


2025-09-08 18:44:02,043 [ERROR] Error in fixed_add
Traceback (most recent call last):
  File "/tmp/ipython-input-7538178.py", line 7, in <cell line: 0>
    result = fixed_add(5, "hello")
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-7538178.py", line 3, in fixed_add
    raise TypeError("Both arguments must be int or float")
TypeError: Both arguments must be int or float




That verbose output is **the full traceback**, which is useful for deep debugging but can feel overwhelming or cluttered — especially in production logs or when you're scanning for known issues.

Let’s break down your options depending on your goal:

---

## 🧼 Option 1: Show Only the Error Message (Cleaner Logging)

If you want a **clean, readable log**, log only the message *without traceback*:

### ✅ Use `logging.error(str(e))`

```python
try:
    result = fixed_add(5, "hello")
except Exception as e:
    logging.error(f"fixed_add failed: {e}")  # just the message, no traceback
```

### Output:

```
2025-09-08 19:00:00 [ERROR] fixed_add failed: Both arguments must be int or float
```

🔍 **Much cleaner.** This is ideal when:

* You already know *where* the error is.
* You want user-friendly logs.
* You don’t need the full traceback.

---

## 🧰 Option 2: Keep Full Traceback (for debugging)

```python
logging.exception("fixed_add failed")  # includes full traceback
```

Best for:

* Early dev/debugging
* Unexpected or unknown errors
* When you’re not sure *where* the error occurred

---

## 🧩 Option 3: Log Both (Clean + Traceback Only for Debug)

You can **log both** — a clean message for regular viewing, and the traceback **only when debugging**:

```python
try:
    result = fixed_add(5, "hello")
except Exception as e:
    logging.error(f"fixed_add failed: {e}")
    logging.debug("Full traceback below:", exc_info=True)
```

Then simply change your logging level to `DEBUG` if you want to see tracebacks later.

---

## 🧙‍♂️ Pro Tip: Use Custom Logger with Filtering

For more control (like saving verbose logs to file but showing clean logs on screen), use **separate loggers or handlers** — we can cover this if you're interested later.

---

### ✅ Recommendation for Now

Since you're learning and building agents:

* Use `logging.exception(...)` when **developing/debugging**
* Use `logging.error(...)` with a clear message when **in production / user-facing logs**
* Combine with `DEBUG` logs if you want selective verbosity





### ✅ `logging.exception(...)` → **Development & Debugging Phase**

🧪 Use this when:

* You are **exploring what could go wrong**
* You're writing or testing new code
* You need the **full traceback** to see *where* and *why* something failed
* You're catching **unexpected** or **generic** errors (`except Exception`)

📋 It logs:

* The message you provide
* The **entire traceback** (file, line, stack context)

🧠 Helps you **investigate** and **learn** from failures

---

### ✅ `logging.error(...)` → **Production & Runtime Phase**

🚀 Use this when:

* You are in a **stable, running environment**
* You expect a specific error to occur (e.g. `ValueError`, `TimeoutError`)
* You want to **log it cleanly** for future review or user support
* You want **simple, readable logs** (especially for external systems or dashboards)

📋 It logs:

* Only the message (unless you explicitly include traceback via `exc_info=True`)
* No noisy stack trace unless needed

🧠 Helps you **track issues** without overwhelming your logs

---

### 🧠 Mental Model

| Phase          | Use                 | Goal                            |
| -------------- | ------------------- | ------------------------------- |
| 🧪 Development | `logging.exception` | Debug deeply, learn what breaks |
| 🚀 Production  | `logging.error`     | Report known issues cleanly     |




### 🧪 Example: Handling File I/O Errors with Logging

This one simulates a common **agent scenario**: reading from a file that might not exist, and handling it gracefully.


### ✅ What You’ll Learn from This Example

* **How to handle expected errors** (`FileNotFoundError`, `PermissionError`)
* How to fall back safely (`return None`)
* How to log **specific vs unexpected** errors
* Why `logging.exception()` is useful for uncaught edge cases




In [6]:
import os

def read_file(path):
    try:
        with open(path, "r") as f:
            return f.read()
    except FileNotFoundError:
        logging.error(f"File not found: {path}")
        return None
    except PermissionError:
        logging.error(f"No permission to read: {path}")
        return None
    except Exception as e:
        logging.exception("Unexpected error while reading file")
        return None

# Simulate a bad file path
bad_path = "non_existent_file.txt"
contents = read_file(bad_path)

if contents is None:
    print("❌ File read failed.")
else:
    print("✅ File read succeeded.")


2025-09-08 18:50:44,191 [ERROR] File not found: non_existent_file.txt


❌ File read failed.




### 📦 Example: Parsing a JSON Response from a Tool/API

Many AI agents rely on tools that return JSON. Sometimes that JSON is malformed, missing keys, or has the wrong structure. Let’s simulate that and handle the errors cleanly.

### 🔍 What to Watch

* Logging will show up in the **notebook output** (and also go to the file).
* You'll see how different types of errors are handled differently:

  * `json.JSONDecodeError` logs a traceback using `logging.exception(...)`
  * `KeyError` logs a simple message using `logging.error(...)`





In [8]:
import json

def parse_tool_response(response_text):
    try:
        data = json.loads(response_text)
        logging.info("Successfully parsed JSON.")

        if "answer" not in data:
            raise KeyError("Missing 'answer' field in response")

        return data["answer"]

    except json.JSONDecodeError as e:
        logging.exception("Failed to decode JSON response")
        return None

    except KeyError as e:
        logging.error(f"Expected key not found: {e}")
        return None


In [10]:
# Valid input
print("Result 1:", parse_tool_response('{"answer": 42}'))

2025-09-08 18:56:25,540 [INFO] Successfully parsed JSON.


Result 1: 42


In [11]:
# Invalid JSON
print("Result 2:", parse_tool_response('{bad json!'))

2025-09-08 18:56:36,205 [ERROR] Failed to decode JSON response
Traceback (most recent call last):
  File "/tmp/ipython-input-3541659297.py", line 5, in parse_tool_response
    data = json.loads(response_text)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/json/decoder.py", line 338, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/json/decoder.py", line 354, in raw_decode
    obj, end = self.scan_once(s, idx)
               ^^^^^^^^^^^^^^^^^^^^^^
json.decoder.JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


Result 2: None


In [12]:
# Valid JSON but missing key
print("Result 3:", parse_tool_response('{"result": "ok"}'))

2025-09-08 18:56:38,883 [INFO] Successfully parsed JSON.
2025-09-08 18:56:38,885 [ERROR] Expected key not found: "Missing 'answer' field in response"


Result 3: None



### 🧩 What Happened

```python
print("Result 3:", parse_tool_response('{"result": "ok"}'))
```

You're passing **valid JSON**, but it doesn't contain the key your agent expects (e.g., `"answer"`). So:

* ✅ `json.loads()` succeeds → `logging.info("Successfully parsed JSON.")`
* ❌ But the `"answer"` field is **missing**
* → Your code catches that, raises/logs a clear error:

```python
logging.error('Expected key not found: "Missing \'answer\' field in response"')
```

* 👇 The function returns `None`, indicating failure — without crashing the agent.

---

### 🧠 Why This Is Powerful (especially in agents):

| Benefit                  | Why it Matters                                                               |
| ------------------------ | ---------------------------------------------------------------------------- |
| ✅ Clear timestamps       | Shows **when** the failure happened — helps identify performance bottlenecks |
| ✅ Separation of concerns | JSON parsing vs key validation errors are distinct — easier to isolate bugs  |
| ✅ Graceful failure       | Instead of crashing, the agent logs the problem and moves on                 |
| ✅ Traceability           | When reviewing logs, you know exactly **which step failed and why**          |
| ✅ Maintainability        | Future you (or a teammate) will **thank you** for this logging clarity       |

---

### 🛠️ You’re Practicing This Principle:

> **"Fail fast, fail clearly, fail gracefully."**

That’s the **gold standard** for agent step tooling.



## 🧪 Example: Fetching from a fake tool (some responses succeed, some fail)

In [16]:
# Fixed responses: last one will fail
responses = [
    '{"answer": 42}',
    '{"answer": 13}',
    '{"answer": 7}',
    '{"answer": 99}',
    '{"result": "ok"}'  # <-- This one is missing the 'answer' key
]

# Core logic
def parse_tool_response(response_text):
    try:
        data = json.loads(response_text)
        logging.info("Successfully parsed JSON.")

        if "answer" not in data:
            raise ValueError("Missing 'answer' field in response")

        answer = data["answer"]

        if not isinstance(answer, int):
            raise TypeError("Answer must be an integer")

        return answer

    except Exception as e:
        logging.error("Failed to parse response: %s", e, exc_info=True)
        return None

# Run each step
for i, resp in enumerate(responses, 1):
    print(f"\n▶️ Run {i}")
    result = parse_tool_response(resp)
    print(f"Result: {result}")


2025-09-08 19:07:00,582 [INFO] Successfully parsed JSON.
2025-09-08 19:07:00,584 [INFO] Successfully parsed JSON.
2025-09-08 19:07:00,586 [INFO] Successfully parsed JSON.
2025-09-08 19:07:00,587 [INFO] Successfully parsed JSON.
2025-09-08 19:07:00,588 [INFO] Successfully parsed JSON.
2025-09-08 19:07:00,589 [ERROR] Failed to parse response: Missing 'answer' field in response
Traceback (most recent call last):
  File "/tmp/ipython-input-1413892704.py", line 17, in parse_tool_response
    raise ValueError("Missing 'answer' field in response")
ValueError: Missing 'answer' field in response



▶️ Run 1
Result: 42

▶️ Run 2
Result: 13

▶️ Run 3
Result: 7

▶️ Run 4
Result: 99

▶️ Run 5
Result: None




The **core purpose of proper error handling** is *not just to catch problems*, but to allow your program or agent to:

* **Continue running**
* **Gracefully handle known issues**
* **Log or report what went wrong**
* **Recover or fallback if possible**

Let’s do an example where some agent tool responses succeed and some fail — but the **agent keeps running**, logging each step, and collecting results where possible.

---

### ✅ Example: Mixed Pass/Fail with Logging, Error Handling, and Graceful Completion


### 🔍 What to Notice

* The agent processes all 5 responses.
* Failures are logged with tracebacks for debugging.
* The process **doesn't crash**, even with invalid JSON or missing fields.
* We return `None` for failed steps, but continue collecting valid outputs.
* Final result shows all values, successful and failed.

---

### 🧠 Why This Is Useful for Agents

* This structure is **very close to how real agents operate**: run a bunch of tool calls → handle failures → gather partial results → keep going.
* **Logging and structured handling** means your agent doesn't get derailed by one bad response.
* You can plug in **retries**, **fallbacks**, or **summarize the failures** for the user in the end.



In [17]:
# Simulated responses: a mix of valid and invalid JSON or structures
responses = [
    '{"answer": 42}',                      # ✅ valid
    '{"answer": "not a number"}',         # ❌ wrong type
    '{"result": "ok"}',                   # ❌ missing key
    'not json at all',                    # ❌ invalid JSON
    '{"answer": 13}'                      # ✅ valid
]

# Function to parse and validate the response
def parse_tool_response(response_text):
    try:
        data = json.loads(response_text)
        logging.info("✅ Parsed JSON successfully.")

        if "answer" not in data:
            raise ValueError("Missing 'answer' field in response.")

        answer = data["answer"]
        if not isinstance(answer, int):
            raise TypeError("Answer must be an integer.")

        return answer

    except Exception as e:
        logging.error("❌ Failed to process response: %s", e, exc_info=True)
        return None

# Process all responses and collect results
results = []

for i, resp in enumerate(responses, 1):
    print(f"\n▶️ Run {i}")
    result = parse_tool_response(resp)
    print(f"Result: {result}")
    results.append(result)

# Final output
print("\n✅ All runs completed.")
print("Collected results:", results)


2025-09-08 19:09:42,633 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:09:42,634 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:09:42,638 [ERROR] ❌ Failed to process response: Answer must be an integer.
Traceback (most recent call last):
  File "/tmp/ipython-input-2823341378.py", line 21, in parse_tool_response
    raise TypeError("Answer must be an integer.")
TypeError: Answer must be an integer.
2025-09-08 19:09:42,640 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:09:42,641 [ERROR] ❌ Failed to process response: Missing 'answer' field in response.
Traceback (most recent call last):
  File "/tmp/ipython-input-2823341378.py", line 17, in parse_tool_response
    raise ValueError("Missing 'answer' field in response.")
ValueError: Missing 'answer' field in response.
2025-09-08 19:09:42,644 [ERROR] ❌ Failed to process response: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/tmp/ipython-input-2823341378.py", line 13, in parse_tool_response
    da


▶️ Run 1
Result: 42

▶️ Run 2
Result: None

▶️ Run 3
Result: None

▶️ Run 4
Result: None

▶️ Run 5
Result: 13

✅ All runs completed.
Collected results: [42, None, None, None, 13]



## 🧱 Step 1: Add the Wrapper

This function wraps your tool call and returns a **standardized result object** like an agent would expect:

```python
def run_step(step_fn):
    try:
        return {"ok": True, "value": step_fn()}
    except Exception as e:
        logging.error("Step failed: %s", e, exc_info=True)
        return {"ok": False, "error": str(e)}
```


## 📌 What You’re Learning

| Feature             | Benefit                                   |
| ------------------- | ----------------------------------------- |
| `run_step(...)`     | Centralizes error handling                |
| `lambda:`           | Delays execution to wrap function call    |
| Structured result   | Easy for agent/controller to reason about |
| Logs with traceback | You know exactly where it failed          |



In [19]:
# Simulated tool responses
responses = [
    '{"answer": 42}',                      # ✅ valid
    '{"answer": "not a number"}',         # ❌ wrong type
    '{"result": "ok"}',                   # ❌ missing key
    'not json at all',                    # ❌ invalid JSON
    '{"answer": 13}'                      # ✅ valid
]

# Structured agent-like wrapper
def run_step(step_fn):
    try:
        return {"ok": True, "value": step_fn()}
    except Exception as e:
        logging.error("Step failed: %s", e, exc_info=True)
        return {"ok": False, "error": str(e)}

# Parser with error handling + logging
def parse_tool_response(response_text):
    data = json.loads(response_text)
    logging.info("✅ Parsed JSON successfully.")

    if "answer" not in data:
        raise ValueError("Missing 'answer' field in response.")

    answer = data["answer"]
    if not isinstance(answer, int):
        raise TypeError("Answer must be an integer.")

    return answer

# Run all responses
results = []

for i, resp in enumerate(responses, 1):
    print(f"\n▶️ Run {i}")
    result = run_step(lambda: parse_tool_response(resp))
    print("Result:", result)
    results.append(result)

# Final output
print("\n✅ All runs completed.")
print("Collected results:", results)


2025-09-08 19:17:12,289 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:17:12,291 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:17:12,292 [ERROR] Step failed: Answer must be an integer.
Traceback (most recent call last):
  File "/tmp/ipython-input-908012165.py", line 13, in run_step
    return {"ok": True, "value": step_fn()}
                                 ^^^^^^^^^
  File "/tmp/ipython-input-908012165.py", line 37, in <lambda>
    result = run_step(lambda: parse_tool_response(resp))
                              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-908012165.py", line 28, in parse_tool_response
    raise TypeError("Answer must be an integer.")
TypeError: Answer must be an integer.
2025-09-08 19:17:12,293 [INFO] ✅ Parsed JSON successfully.
2025-09-08 19:17:12,294 [ERROR] Step failed: Missing 'answer' field in response.
Traceback (most recent call last):
  File "/tmp/ipython-input-908012165.py", line 13, in run_step
    return {"ok": True, "value": step_fn()}
     


▶️ Run 1
Result: {'ok': True, 'value': 42}

▶️ Run 2
Result: {'ok': False, 'error': 'Answer must be an integer.'}

▶️ Run 3
Result: {'ok': False, 'error': "Missing 'answer' field in response."}

▶️ Run 4
Result: {'ok': False, 'error': 'Expecting value: line 1 column 1 (char 0)'}

▶️ Run 5
Result: {'ok': True, 'value': 13}

✅ All runs completed.
Collected results: [{'ok': True, 'value': 42}, {'ok': False, 'error': 'Answer must be an integer.'}, {'ok': False, 'error': "Missing 'answer' field in response."}, {'ok': False, 'error': 'Expecting value: line 1 column 1 (char 0)'}, {'ok': True, 'value': 13}]




## ✅ Next Concepts to Learn (Post-Error Handling & Logging)

### 1. **Retries with Backoff**

* **Why?** Many agent tools fail due to *transient errors* (timeouts, flaky APIs).
* Learn how to:

  * Retry only certain exceptions
  * Use exponential backoff (`2^n` delays)
  * Cap retries to avoid infinite loops
* 🔁 Pairs perfectly with error handling and structured return values.

> **Try next:** Wrap one of your failing responses in a retry mechanism.

---

### 2. **Structured Error Returns (`{ok, error, retryable}`)**

* **Why?** So the **agent controller** (or outer logic) can make intelligent choices:

  * Retry if it’s a timeout
  * Skip if it’s a user mistake
  * Fallback if something is missing

> **Try next:** Modify your last example to return structured results instead of `None`.

---

### 3. **Top-Level Agent Step Wrappers**

* Build a reusable function like:

```python
def run_step(step_fn):
    ...
    return {"ok": True, "value": ...}
```

* This **standardizes** error behavior across all tools — so your agent only needs to reason about a **single format**.

> You’ve already explored this, but now’s the time to **implement and reuse it** in your own tool calls.

---

### 4. **Input Validation Before Expensive Work**

* Before calling tools or APIs, check:

  * Type? ✅
  * Empty string? ❌
  * Reasonable range? ✅

> Keeps bugs *upstream* and avoids confusing downstream errors.

---

### 5. **Testing Agent Code**

* Use `pytest` to:

  * Ensure invalid input throws expected errors
  * Validate that retry wrappers retry on transient errors
  * Confirm logs are generated properly (advanced)

> Helps keep your agent components solid as they grow.

---

### 6. **Logging to Separate Files / Levels**

* Separate logs by:

  * `agent_steps.log`
  * `tool_errors.log`
* Adjust levels: `DEBUG`, `INFO`, `WARNING`, `ERROR`

> Makes logs easier to sift through in production.

---

### 7. **Agent Recovery Patterns**

* Build flows where the agent:

  * Tries Tool A
  * On failure, logs & tries Tool B
  * On complete failure, summarizes error for user

> This is a **hallmark of resilient agents** — graceful degradation.

---

### Optional (Advanced):

* **Async / concurrency patterns** (e.g., `asyncio.gather`)
* **Observability**: collecting metrics from logs
* **Contextual logging** (add step ID, user, etc.)

---

## 🧭 Want a Learning Roadmap?

Here’s a simplified sequence:

| Stage       | Focus Area                                                 |
| ----------- | ---------------------------------------------------------- |
| ✅ Now       | Error handling, logging basics                             |
| 🔁 Next     | Retry logic, structured returns                            |
| 🧠 After    | Step wrappers, validation                                  |
| 🧪 Advanced | Testing, recovery, async                                   |
| 🛠️ Tooling | Custom error classes, logging configs, agent control loops |



This is one of the key mindset shifts as you move from **prototyping** to **production-grade development**.

---

## ✅ Yes, `logging` can (and often *should*) be used **instead of** `print`.

But the choice depends on **context**:

---

### 🔁 `print()` is:

* **Quick & dirty**
* Meant for **temporary debugging or local exploration**
* **Always writes to stdout** — you can't control where it goes
* Has **no severity level** (you can't tell what's a warning vs info vs error)
* Gets messy when working in larger codebases, threads, or background jobs

---

### 🛠️ `logging` is:

* **Built for real debugging, observability, and tracing**
* Lets you assign **levels**: `DEBUG`, `INFO`, `WARNING`, `ERROR`, `CRITICAL`
* Can write to **multiple places**: file, stdout, remote server, etc.
* You can **turn it on/off**, **filter**, and **format** it consistently
* Works great with **concurrent code**, **agent tools**, **background workers**, etc.
* Provides full **timestamps**, **tracebacks**, and **error categorization**

---

### ✅ In Agent Development:

Use `logging` to:

* Track what tools your agent is using
* Measure how long steps take
* Record when/why something failed
* Debug retry behavior, concurrency bugs, or timeouts
* Build traceable, production-safe systems

---

### 📌 Quick Example:

#### Using `print`:

```python
print("Calling API")
response = api_call()
print("Response:", response)
```

#### Using `logging` (preferred):

```python
logging.info("Calling API")
response = api_call()
logging.debug(f"Raw API response: {response}")
```

---

### 🧪 Bonus Tip:

You can even set **different levels** depending on the audience:

* `DEBUG`: for you, the developer
* `INFO`: for general operation logging
* `WARNING/ERROR`: for things that went wrong but didn’t crash
* `CRITICAL`: for actual crashes or halts

---

Would you like a side-by-side comparison in a Colab-style example so you can see how `print` vs `logging` behaves?


## View Error Logs

In [18]:
# 📄 To View the Logs Afterward:
def show_logs():
    with open("agent_log.txt") as f:
        print(f.read())

show_logs()

2025-09-08 18:43:23,950 [ERROR] Error in buggy_add
Traceback (most recent call last):
  File "/tmp/ipython-input-945670281.py", line 5, in <cell line: 0>
    result = buggy_add(5, "hello")  # TypeError!
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-945670281.py", line 2, in buggy_add
    return x + y
           ~~^~~
TypeError: unsupported operand type(s) for +: 'int' and 'str'
2025-09-08 18:44:02,043 [ERROR] Error in fixed_add
Traceback (most recent call last):
  File "/tmp/ipython-input-7538178.py", line 7, in <cell line: 0>
    result = fixed_add(5, "hello")
             ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-7538178.py", line 3, in fixed_add
    raise TypeError("Both arguments must be int or float")
TypeError: Both arguments must be int or float
2025-09-08 18:50:44,191 [ERROR] File not found: non_existent_file.txt
2025-09-08 18:56:17,138 [INFO] Successfully parsed JSON.
2025-09-08 18:56:17,140 [ERROR] Failed to decode JSON response
Traceback (most recent c